# DataJoint U24 - Workflow Session

This notebook will describe the steps to explore the lab and animal management tables created by the elements.
Prior to using this notebook, please refer to the README for the installation instructions.

## Testing load function

In [3]:
# change to the upper level folder to detect dj_local_conf.json
import os
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
import datajoint as dj
dj.conn()

Please enter DataJoint username:  root
Please enter DataJoint password:  ········


Connecting root@localhost:3306


DataJoint connection (connected) root@localhost:3306

In [40]:
import pathlib
from workflow_trial.pipeline import subject, session, trial
from workflow_trial.paths import get_trial_root_dir
from element_data_loader.utils import find_full_path
dj.config['custom'] = {
        "root_data_dir": [
            "/Users/cb/Documents/U24_SampleData/workflow_trial_data1/",
            "/Users/cb/Documents/U24_SampleData/workflow_trial_data2/"
        ]}
from datajoint.user_tables import UserTable
import inspect
def list_tables(schema):
  for k in dir(schema):
     t = getattr(schema, k)
     if inspect.isclass(t) and issubclass(t, UserTable):
        print(k)

In [21]:
from workflow_trial.ingest import ingest_subjects, ingest_sessions
ingest_subjects(); _ = ingest_sessions()

In [122]:
bpods_filepath=["Bpod_Behavior_Data/BalbC_Ph_W1_LH_Randdelay_changeover_Aug08_2021_Session1.mat",
    "Bpod_Behavior_Data/1119A_3_LoomingThreat_Dec26_2019_Session1.mat",
    "Bpod_Behavior_Data_and_TrialEvents/NeuroPixels/TQ03_Dual2AFC_Jun18_2021_Session1.mat",
    # "Bpod_Behavior_Data_and_TrialEvents/NeuroPixels/TrialEvents.mat",
    "Bpod_Behavior_Data_and_TrialEvents/Neuralynx/TP24_Dual2AFC_Feb12_2019_Session1.mat",
    # "Bpod_Behavior_Data_and_TrialEvents/Neuralynx/TrialEvents.mat"
               ]
# taking first
bpod_filepath = find_full_path(get_trial_root_dir(),bpods_filepath[0])
print(bpod_filepath)

/Users/cb/Documents/U24_SampleData/workflow_trial_data1/Bpod_Behavior_Data/BalbC_Ph_W1_LH_Randdelay_changeover_Aug08_2021_Session1.mat


In [188]:
import scipy.io as spio
def load_bpod_matfile(matlab_filepath):
    """
    Loading routine for behavioral file, bpod .mat
    """
    SessionData = spio.loadmat(matlab_filepath.as_posix(),
                               squeeze_me=True,simplify_cells=True,
                               struct_as_record=False)['SessionData']
    return SessionData

In [199]:
Standard = ['RawEvents','RawData','TrialSettings','nTrials','TrialStartTimestamp','Settings','TrialTypes']
for f in bpods_filepath[1:]:
    filepath = find_full_path(get_trial_root_dir(),f)
    print(str(filepath).split("/")[-1])
    data = load_bpod_matfile(filepath)
    for field in list(data.keys()):
        if field not in Standard: 
            print(field)
            # print(data[field])

1119A_3_LoomingThreat_Dec26_2019_Session1.mat
Custom
Notes
MarkerCodes
TQ03_Dual2AFC_Jun18_2021_Session1.mat
Custom
TP24_Dual2AFC_Feb12_2019_Session1.mat
Custom


In [217]:
#check if TrialSettings vals the same across trials
for f in bpods_filepath:
    filepath = find_full_path(get_trial_root_dir(),f)
    print(str(filepath).split("/")[-1])
    data = load_bpod_matfile(filepath)
    compare = data['TrialSettings'][0]['GUI']
    compare_keys = list(compare.keys())
    for n in range(1,10):#data.nTrials):
        thisone = data['TrialSettings'][n]['GUI']
        for k in compare_keys:
            try: 
                if compare[k] != thisone[k]: print(k, thisone[k])
            except ValueError: pass

BalbC_Ph_W1_LH_Randdelay_changeover_Aug08_2021_Session1.mat
RewardAmount nan
RewardAmount nan
RewardAmount nan
RewardAmount nan
RewardAmount nan
RewardAmount nan
RewardAmount nan
RewardAmount nan
RewardAmount nan
1119A_3_LoomingThreat_Dec26_2019_Session1.mat
TQ03_Dual2AFC_Jun18_2021_Session1.mat
FeedbackDelay 0.6268574714495521
StimDelay 0.2944547748784063
FeedbackDelay 0.7178875499064047
StimDelay 0.27178970504944655
FeedbackDelay 2.8843670003984934
StimDelay 0.3851048369726737
FeedbackDelay 2.1407255446877764
StimDelay 0.29875678993380594
FeedbackDelay 1.1578884707983916
StimDelay 0.37483520678206617
FeedbackDelay 5.601414410662132
StimDelay 0.20663509454288095
FeedbackDelay 1.9234691430710023
StimDelay 0.2530836105527989
FeedbackDelay 2.0582781374115378
StimDelay 0.22281014275234973
FeedbackDelay 4.698232160540173
StimDelay 0.3419607784649046
TP24_Dual2AFC_Feb12_2019_Session1.mat
FeedbackDelay 0.6834884662550295
MinSampleAud 0.35
StimDelay 0.2754547863593935
FeedbackDelay 1.76546998

In [223]:
# see trialtype arrays
for f in bpods_filepath:
    filepath = find_full_path(get_trial_root_dir(),f)
    print(str(filepath).split("/")[-1])
    data = load_bpod_matfile(filepath)
    try: print(data['TrialTypes'])
    except KeyError: pass

BalbC_Ph_W1_LH_Randdelay_changeover_Aug08_2021_Session1.mat
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1]
1119A_3_LoomingThreat_Dec26_2019_Session1.mat
[1 1 1 1 1 1 1 1 3 1 1 1 1 3 1 1 1 1 1 3 1 1 1 1 3 1 1 1 1 1 2 1 1 1 3 1 1
 1 2 1 1 1 1 1 2 1 1 1 2 1 1 1 3 1 1 1 1 3 1 1 1 3 1 1 1 2 1 1 1 2 1 1 1 1
 2 1 1 1 1 1 3 1 1 1 1 3 1 1 1 1 3 1 1 1 1 1 2 1 1 1 3 1 1 1 2 1 1 1 1 1 2
 1 1 1 1 1 3 1 1 1 1 2 1 1 1 1 2 1 1 1 3 1 1 1 1 3 1 1 1 1 1 2 1 1 1 3 1 1
 1 1 2 1 1 1 2 1 1 1 2 1 1 1 1 3 1 1 1 3 1 1 1 3 1 1 1 3 1 1 1 1 1 3 1 1]
TQ03_Dual2AFC_Jun18_2021_Session1.mat
TP24_Dual2AFC_Feb12_2019_Session1.mat


In [239]:
temp = load_bpod_matfile(bpod_filepath)
temp['RawEvents']['Trial'][0]

{'States': {'WaitForPoke': array([0.    , 0.4043]),
  'LeftRewardDelay': array([0.4043, 0.4044]),
  'LeftReward': array([0.4044, 0.4371]),
  'Drinkingleft': array([0.4371, 6.4371]),
  'RightRewardDelay': array([nan, nan]),
  'RightReward': array([nan, nan]),
  'Drinkingright': array([nan, nan])},
 'Events': {'Port1Out': array([0.374 , 0.4064, 0.4318, 1.1153, 1.1857, 2.8942, 2.9757, 3.0841]),
  'Port1In': array([0.4043, 0.4218, 0.449 , 1.1315, 2.7218, 2.9481, 3.0511]),
  'Tup': array([0.4044, 0.4371, 6.4371]),
  'Port3In': 6.4193999999999996}}

## Workflow architecture

Importing the module `workflow_trial.pipeline` is sufficient to create tables inside the elements. This workflow comes prepackaged with example data and ingestion functions to populate subject, session, trialized or event tables.

In [ ]:
lab.Lab()

In [ ]:
dj.Diagram(lab)

In [ ]:
subject.Subject()

In [ ]:
dj.Diagram(subject)

In [ ]:
session.Session()

In [ ]:
dj.Diagram(session)

## Explore each table

In [ ]:
# check table definition with describe()
subject.Subject.describe()

## Insert data into Manual and Lookup tables

Tables in this workflow are either manual tables or lookup tables. To insert into these tables, DataJoint provide method `.insert1()` and `insert()`.

In [ ]:
subject.Subject.insert1(
    dict(subject='subject1', sex='M', subject_birth_date='2020-12-30', 
         subject_description='test animal'), skip_duplicates=True)
subject.Subject.insert1(
    ('subject2', 'F', '2020-11-30', 'test animal'), skip_duplicates=True)

`skip_duplicates=True` will prevent an error if you already have data for the primary keys in a given entry.

In [ ]:
subject.Subject()

In [ ]:
# `insert()` takes a list of dicts or tuples
subject.Subject.insert(
    [dict(subject='subject3', sex='F', subject_birth_date='2020-12-30', 
            subject_description='test animal'),
     dict(subject='subject4', sex='M', subject_birth_date='2021-02-12', 
          subject_description='test animal')
    ],
    skip_duplicates=True)
subject.Subject.insert(
    [
        ('subject7', 'U', '2020-08-30', 'test animal'),
        ('subject8', 'F', '2020-09-30', 'test animal')
    ],
    skip_duplicates=True)

In [ ]:
subject.Subject()

For more documentation of insert, please refer to [DataJoint Docs](https://docs.datajoint.io/python/manipulation/1-Insert.html) and [DataJoint playground](https://playground.datajoint.io/)

## Insert into Manual and Lookup tables with Graphical User Interface

DataJoint also provides a Graphical User Interface [DataJoint Labbook](https://github.com/datajoint/datajoint-labbook) to support manual data insertions into DataJoint workflows. ![DataJoint Labbook preview](../images/DataJoint_Labbook.png)